In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import os
import numpy as np
import pandas as pd
import os
from music21 import converter, note, chord, instrument, stream
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

In [19]:
# Define the folder path containing MIDI files
folder_path = 'C:/Users/devil/181 FINAL PROJ/test/The_Who'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define an empty list to store all the notes
notes = []

# Loop through all the MIDI files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.mid'):
        # Load the MIDI file
        midi_file = converter.parse(os.path.join(folder_path, file))
        
        # Extract notes and chords from the MIDI file
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi_file)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi_file.flat.notes
        
        # Append notes and chords to the notes list
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                notes.append('.')
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

# Create a dictionary mapping notes and chords to integers
note_to_int = {}
int_to_note = {}
for i, note in enumerate(sorted(set(notes))):
    note_to_int[note] = i
    int_to_note[i] = note

# Convert notes to integers
note_ints = [note_to_int[note] for note in notes]

# Convert integers to characters
chars = [int_to_note[i] for i in note_ints]

# Convert list of characters to a string
data = ''.join(chars)
print(data)

0.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.107.07.07.07.07.00.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.105.9.05.9.05.9.05.9.05.9.00.57.05.100.57.05.100.57.05.100.57.05.107.07.05.105.107.05.100.510.37.05.100.510.3B3.B3.B3.A3.G3.B3.B3.B3.B3.B3.A3.F#3.F#3.F#3.B3.B3.A3.E3.E3.G3.A3.A3.B3.A3.A3.B3.G4.B3.G4.B3.B4.A3.A4.G3.G4.B3.E5.B3.D5.B3.G4.B3.B4.B3.D5.A3.A4.F#3.D5.F#3.D5.F#3.F#4.B3.B4.B3.D5.A3.A5.E3.E5.E3.E5.D3.D4.E3.E4.G3.G4.A3.A4.A3.A4.B3.B4.A3.A4.A3.A4.B4.D5.E5.D5.B4.A4.G4.E4.E4.E4.G4.A4.A4.A4.F#4.F#4.G4.F#4.D4.E5.B3.B4.B3.B4.B3.B4.A3.A4.G3.G4.B3.B4.B3.B4.B3.B4.B3.B4.B3.B4.A3.A4.F#3.F#4.F#3.F#4.F#3.F#4.B3.B4.B3.B4.A3.A4.E3.E4.E3.E4.G3.G4.A3.A4.A3.A4.B3.B4.A3.A4.A3.A4.B3.G4.B3.G4.B3.B4.A3.A4.G3.G4.B3.E5.B3.D5.B3.G4.B3.B4.B3.D5.A3.A4.F#3.D5.F#3.D5.F#3.F#4.B3.B4.B3.D5.A3.A5.E3.E5.E3.E5.D3.D4.E

In [20]:
# Define the sequence length
seq_length = 100

# Split the data into sequences
sequences = []
for i in range(0, len(data) - seq_length, 1):
    seq_in = data[i:i + seq_length]
    seq_out = data[i + seq_length]
    sequences.append((seq_in, seq_out))

In [21]:
# Get all the unique characters in the music data
all_chars = set(data)

# Define a dictionary to map characters to integers
char_to_int = {}
char_to_int[' '] = 0
for i, char in enumerate(sorted(all_chars)):
    char_to_int[char] = i + 1

# Define a dictionary to map integers back to characters
int_to_char = {}
for char, i in char_to_int.items():
    int_to_char[i] = char

# Convert sequences to sequences of integers
X = []
y = []
for seq_in, seq_out in sequences:
    X.append([char_to_int[char] for char in seq_in])
    y.append(char_to_int[seq_out])

In [22]:
# One-hot encode the data
num_chars = len(char_to_int)
X_one_hot = np.zeros((len(X), seq_length, num_chars), dtype=np.bool)
y_one_hot = np.zeros((len(y), num_chars), dtype=np.bool)
for i, seq in enumerate(X):
    for j, char in enumerate(seq):
        X_one_hot[i, j, char] = 1
    y_one_hot[i, y[i]] = 1

C:\Users\devil\AppData\Local\Temp\ipykernel_13284\2641926062.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_one_hot = np.zeros((len(X), seq_length, num_chars), dtype=np.bool)
C:\Users\devil\AppData\Local\Temp\ipykernel_13284\2641926062.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_one_hot = np.zeros((len(y), num_chars), dtype=np.bool

In [24]:
# Define the model
with tf.device('/GPU:0'):  # specify the GPU device
    model = Sequential()
    model.add(LSTM(128, input_shape=(seq_length, num_chars)))
    model.add(Dense(num_chars, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_one_hot, y_one_hot, test_size=0.2, random_state=42)
# Train the model on the training set
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
98/98 [==============================] - 11s 101ms/step - loss: 2.2611 - val_loss: 2.0683
Epoch 2/50
98/98 [==============================] - 10s 100ms/step - loss: 1.8083 - val_loss: 1.5807
Epoch 3/50
98/98 [==============================] - 10s 100ms/step - loss: 1.1933 - val_loss: 1.0131
Epoch 4/50
98/98 [==============================] - 10s 100ms/step - loss: 0.9250 - val_loss: 0.8824
Epoch 5/50
98/98 [==============================] - 10s 99ms/step - loss: 0.8367 - val_loss: 0.8107
Epoch 6/50
98/98 [==============================] - 10s 99ms/step - loss: 0.7913 - val_loss: 0.7601
Epoch 7/50
98/98 [==============================] - 10s 99ms/step - loss: 0.7517 - val_loss: 0.7368
Epoch 8/50
98/98 [==============================] - 10s 98ms/step - loss: 0.7178 - val_loss: 0.7078
Epoch 9/50
98/98 [==============================] - 10s 99ms/step - loss: 0.6888 - val_loss: 0.6830
Epoch 10/50
98/98 [==============================] - 10s 99ms/step - loss: 0.6620 - val_loss: 0.

In [25]:
# Define a function to generate new music
def generate_music(seed, length):
    # Convert the seed to a sequence of integers
    seed_seq = [char_to_int[char] for char in seed]
    
    # Generate new music
    generated = seed
    for i in range(length):
        x = np.zeros((1, seq_length, num_chars))
        for j, char_int in enumerate(seed_seq):
            x[0, j, char_int] = 1
        
        # Predict the next character
        preds = model.predict(x)[0]
        next_char_int = np.argmax(preds)
        next_char = int_to_char[next_char_int]
        
        # Append the next character to the generated music
        generated += next_char
        
        # Update the seed sequence
        seed_seq.append(next_char_int)
        seed_seq = seed_seq[1:]
        
    return generated

In [28]:
seed = "0.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100."
generated_music = generate_music(seed, 1000)
print(generated_music)

1/1 [==============================] - 0s 9ms/step
0.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.57.05.100.2222224EEEE1.111.4444444144444..444.4.4.44.44.4..........555..8.3..0.52.0..17..5...AAA.A1.....13.1.9.5.5.5..23...3E3...E..7...551.E55....E..33.77.7.775...........2.0200225544.40....A...8.1............5.........113E...00.7..01...5351..5A44...113.99.6..............8.9..48881.......E......5B744.444E4.44.4.44.44.4.4.444444.4.4.4...........3..........23BE.0....44E414444.4..444448.4844444444E.44E4.444E44E4.48.44A.......3................0.E.E.E..11E11E4411111..4.44.8.E.4444.44.A4.A..444..44444444444444444.12448.4....4....4.44.44.4.4.4.4.4.8.4.4.4.4.4.4.4.4.4.4.4.4.4.4.44448.44448.44484.8444E......4.F444444..4.EE8.1494.44.F8..4.4.44..F..4.4..4444444.4.4.4.4444444.4.4.44.0..9.4444.4.444844.4.444E..6.44.44.8..4....4.44.44.444...9..9E.............G.0EEG.EEEE111.11..................4E.033335.5..5.4..5...EE3..3..22444.44..44.4..44FFFFEE.FF.4.444444444..A4.4.4E8.84.4.44444444448

In [29]:
# Define a function to convert the generated music to a MIDI file and save it to disk
def save_music_to_midi(generated_music, output_file):
    # Create a new stream object to hold the generated music
    generated_stream = stream.Stream()
    
    # Parse the generated music string and add notes to the stream
    for element in generated_music.split():
        try:
            # Try to convert the element to a float (for duration)
            duration = float(element)
            note_obj = note.Rest(quarterLength=duration)
        except ValueError:
            # If the conversion fails, assume it's a pitch name
            if "/" in element:
                pitch_name, octave = element.split("/")
                pitch = pitch_name + str(int(octave) + 4)  # Shift the pitch up by 4 octaves
                note_obj = note.Note(pitch)
            else:
                # If the element is not a pitch name or a valid duration, skip it
                continue

        # Add the note/rest to the stream
        generated_stream.append(note_obj)
    
    # Add an instrument to the stream
    instrument_obj = instrument.Piano()
    generated_stream.insert(0, instrument_obj)
    
    # Check if the generated stream contains any notes
#     has_notes = any(isinstance(element, note.Note) for element in generated_stream.flat.notes)

#     if not has_notes:
#         print("Generated music stream is empty. Exiting...")
#         return
    
    # Convert the stream to a MIDI file and save it to disk
    generated_stream.write('midi', fp=output_file)
    print(generated_stream)
    print(f"Generated music saved to {output_file}!")

In [30]:
output_file = "C:/Users/devil/181 FINAL PROJ/output/music.mid"
save_music_to_midi(generated_music, output_file)

<music21.stream.Stream 0x2b4b350d360>
Generated music saved to C:/Users/devil/181 FINAL PROJ/output/music.mid!


In [ ]:
for element in generated_music.split():
    print(element)